### Add scripts path to the notebook

In [83]:
import sys
import os

current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

scripts_path = os.path.join(parent_dir, 'scripts')

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# Insert the path to the Scripts directory
sys.path.insert(0, scripts_path)

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

d:\KifiyaAIM-Course\Week - 2\TellCo-Profitability-Opportunity-Analysis\notebooks


### Import Statements

In [84]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scripts.database_client import DB_Client

### Initialize the DB_client

In [85]:
# obtain values form environment variables
host = os.getenv("DB_HOST")
user_name = os.getenv("DB_USER")
passowrd = os.getenv("DB_PASSWORD")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")

In [86]:
db_client = DB_Client(
    host=host,
    user_name=user_name,
    password=passowrd,
    port=port,
    database_name=database
)

### Load the data from the database

In [87]:
data = db_client.dump_data()

d:\KifiyaAIM-Course\Week - 2\TellCo-Profitability-Opportunity-Analysis\scripts\database_client.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  response = pd.read_sql_query(sql=query, con=self.connection)


Print out basic description of the data

In [88]:
data.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,...,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,...,5.833501e+06,5.517965e+06,5.777156e+06,5.475981e+06,2.104733e+08,4.128476e+06,2.101869e+08,4.145943e+06,3.322201e+07,2.431068e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,45.000000,5.000000,63.000000,...,1.161602e+07,1.101345e+07,1.164222e+07,1.099638e+07,4.234081e+08,8.291208e+06,4.218030e+08,8.267071e+06,4.114331e+07,4.558411e+08
75%,1.304243e+19,749.000000,750.000000,1.324302e+05,2.082018e+14,3.368349e+10,8.611970e+13,70.000000,15.000000,19710.750000,...,1.744852e+07,1.651556e+07,1.747048e+07,1.650727e+07,6.331742e+08,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657055e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


### Data cleaning

##### Handle Missing values

* Count missing values for each column

In [89]:
data.isna().mean()

Bearer Id                                   0.006607
Start                                       0.000007
Start ms                                    0.000007
End                                         0.000007
End ms                                      0.000007
Dur. (ms)                                   0.000007
IMSI                                        0.003800
MSISDN/Number                               0.007107
IMEI                                        0.003813
Last Location Name                          0.007687
Avg RTT DL (ms)                             0.185525
Avg RTT UL (ms)                             0.185412
Avg Bearer TP DL (kbps)                     0.000007
Avg Bearer TP UL (kbps)                     0.000007
TCP DL Retrans. Vol (Bytes)                 0.587636
TCP UL Retrans. Vol (Bytes)                 0.644322
DL TP < 50 Kbps (%)                         0.005027
50 Kbps < DL TP < 250 Kbps (%)              0.005027
250 Kbps < DL TP < 1 Mbps (%)               0.

We can see that there are columns with a lot of missing data (50% - 86% of the instances), I could drop them now but for the current experiment I want to focus on the columns related to the users and those columns aren't related to user. The columns I want to focus on are:

- Handset Manufacturer
- Bearer Id 
- Start
- Start ms
- End
- End ms
- Dur. (ms)
- Dur. (ms).1
- Dur. (s)

In [90]:
columns_of_interest = ["Handset Type", "Handset Manufacturer", "Bearer Id", "Start", "Start ms", "End", "End ms", "Dur. (ms)", "Dur. (ms).1"]
data[columns_of_interest].isna().mean()

Handset Type            0.003813
Handset Manufacturer    0.003813
Bearer Id               0.006607
Start                   0.000007
Start ms                0.000007
End                     0.000007
End ms                  0.000007
Dur. (ms)               0.000007
Dur. (ms).1             0.000007
dtype: float64

I have decided to drop the missing values found in the given column because the amount of data we loose is very little

In [91]:
# look for na values in columns except for 'Handset Manufacturer' and 'Bearer Id' and drop the row if it has one na
data.dropna(subset=columns_of_interest, inplace=True)

Let us check the results now

In [92]:
data[columns_of_interest].isna().mean()

Handset Type            0.0
Handset Manufacturer    0.0
Bearer Id               0.0
Start                   0.0
Start ms                0.0
End                     0.0
End ms                  0.0
Dur. (ms)               0.0
Dur. (ms).1             0.0
dtype: float64

##### Handle Duplicate values

* Let us find the amount of duplicate data

In [93]:
duplicate_data = data[data.duplicated()]

In [94]:
duplicate_data

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)


We can see that there aren't any duplicate data. So nothing to do here

### Exploratory Data Analysis

1) Let us find the top 10 handsets used by the customers 

In [95]:
# group the dataset by the Handset Type column
handset_grouping = data.groupby(by="Handset Type")
# count the rows belonging to each type and then order it in descending order
handset_count = handset_grouping.size().reset_index(name="User Count").sort_values(ascending=False, by="User Count")

In [96]:
# print the top 10 handsets used by the customers
handset_count.head(10)

,Handset Type,User Count
315,Huawei B528S-23A,19752
53,Apple iPhone 6S (A1688),9401
49,Apple iPhone 6 (A1586),9004
1392,undefined,8960
59,Apple iPhone 7 (A1778),6301
73,Apple iPhone Se (A1723),5176
66,Apple iPhone 8 (A1905),4985
78,Apple iPhone Xr (A2105),4562
953,Samsung Galaxy S8 (Sm-G950F),4504
75,Apple iPhone X (A1901),3809


2) Let us find the top 3 handset manufacturers

In [100]:
# group the dataset by the Handset Manufacturer column
manufacturer_grouping = data.groupby(by="Handset Manufacturer")
# count the rows belonging to each manufacturer and then order it in descending order
manufacturer_count = manufacturer_grouping.size().reset_index(name="User Count").sort_values(ascending=False, by="User Count")

In [109]:
manufacturer_count.head(3)

Index(['Handset Manufacturer', 'User Count'], dtype='object')

3) Let us find the top 3 handsets per top 3 manufacturer

In [118]:
# Step 1: Group the data by 'Handset Manufacturer' and 'Handset Type', and count occurrences
handset_counts = data.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')

# Step 2: Get the top 3 manufacturers by total count
top_3_manufacturers = handset_counts.groupby('Handset Manufacturer')['Count'].sum().nlargest(3).index

# Step 3: Filter data for only the top 3 manufacturers
filtered_handset_counts = handset_counts[handset_counts['Handset Manufacturer'].isin(top_3_manufacturers)]

# Step 4: For each manufacturer, get the top 5 handset types
top_handsets_per_manufacturer = filtered_handset_counts.groupby('Handset Manufacturer').apply(
    lambda x: x.nlargest(5, 'Count')
).reset_index(drop=True)

# Display the result
print(top_handsets_per_manufacturer)


   Handset Manufacturer                    Handset Type  Count
0                 Apple         Apple iPhone 6S (A1688)   9401
1                 Apple          Apple iPhone 6 (A1586)   9004
2                 Apple          Apple iPhone 7 (A1778)   6301
3                 Apple         Apple iPhone Se (A1723)   5176
4                 Apple          Apple iPhone 8 (A1905)   4985
5                Huawei                Huawei B528S-23A  19752
6                Huawei                    Huawei E5180   2079
7                Huawei  Huawei P20 Lite Huawei Nova 3E   2015
8                Huawei                      Huawei P20   1476
9                Huawei                  Huawei Y6 2018    996
10              Samsung    Samsung Galaxy S8 (Sm-G950F)   4504
11              Samsung      Samsung Galaxy A5 Sm-A520F   3716
12              Samsung     Samsung Galaxy J5 (Sm-J530)   3689
13              Samsung     Samsung Galaxy J3 (Sm-J330)   3476
14              Samsung    Samsung Galaxy S7 (Sm-G930X)

C:\Users\VICTUS 16\AppData\Local\Temp\ipykernel_15528\103669832.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_handsets_per_manufacturer = filtered_handset_counts.groupby('Handset Manufacturer').apply(
